In [1]:
from plasma_classes import *
from plasma_utils import *
import numpy as np
from matplotlib import pyplot as plt

In [2]:
L = 0.01
N_x = 100
N_p = 10000
h = L/N_x
nodes_grid = np.linspace(0, L, N_x + 1)
particles_grid = np.linspace(0, L, N_p + 1)
n0 = 1e17
n1 = n0*N_x/N_p
q = 1.60217e-19
m = 9.1093837e-31
epsilon = 8.85e-12
particles = Particles(N_p, n1, -q, m)
nodes = Nodes(N_x)
particles.x = (np.sin(particles_grid*1000)*L)/2+L/2
particles.normalise(h, 1)

In [3]:
for i in range(1000):
    get_rho(nodes, particles)

In [4]:
for i in range(1000):
    get_rho_opt(nodes, particles)

In [ ]:
def get_rho(nodes: Nodes, particles, periodic=False):
    """
    Obtains rho value in the nodes using 1-order weighting
    params:
    nodes: spatial grid of nodes
    particles_tpl: set or tuple of sets of physical macroparticles
    periodic: defines boundary conditions
    """
    conc = np.zeros(nodes.length, dtype=np.double)

    x_j = np.floor(particles.x).astype(int)
    x_jplus1 = x_j + 1

    left = particles.concentration * (x_jplus1 - particles.x)
    right = particles.concentration * (particles.x - x_j)

    np.add.at(conc, x_j, left)
    np.add.at(conc, x_jplus1, right)

    if periodic:
        np.add.at(conc, np.where(x_j == 0)[0][-1], left[x_j == 0])
        np.add.at(conc, np.where(x_jplus1 == nodes.length - 1)[0][0], right[x_jplus1 == nodes.length - 1])

    if particles.q > 0:
        nodes.conc_i += conc
    else:
        nodes.conc_e += conc

    rho = conc * particles.q
    np.copyto(nodes.rho, nodes.rho + rho, where=rho != 0)